# Database Assembly using the LMRv2.1 pickle file

## Author
[Tanaya Gondhalekar](https://orcid.org/0009-0004-2440-3266)

This notebook walks through the process of building a [`cfr` ProxyDatabase object](https://github.com/fzhu2e/cfr/blob/1a869ae73648ee6722372190c150deee17e31415/cfr/proxy.py#L1027) containing only the PAGES 2k (version 2.0.0) proxy records. For reproducibility purposes, our starting point here are the [proxy pickle files](LMR_data.tar.gz) from the [LMR Project Data Page](https://atmos.washington.edu/%7Ehakim/LMR/docs/install.html#sample-data) provided in [Tardif et al. (2019)](https://doi.org/10.5194/cp-15-1251-2019), from which we clean and reformat the records to be compatible with the `cfr` data assimilation workflow.

Key steps include:
- Filtering for PAGES2kv2 records
- Standardizing proxy record identifiers (short `pid`)
- Merging proxy time and values with metadata
- Assigning standardized `ptype` labels used by `cfr`
- Filtering for annual and subannual records with sufficient temporal coverage

The final result is saved as a NetCDF file that can be used in subsequent data assimilation notebooks.


In [1]:
import cfr
import pandas as pd
import numpy as np
import pyleoclim as pyleo
import matplotlib.pyplot as plt

octave not found, please see README


## Combining metadata and data pickle files 

### Loading metadata and data csv file and filtering only PAGES2kv2 data

When the raw pickle file (one for metadata, one for data) for the proxies is downloaded from the [LMR Project Data Page ](https://atmos.washington.edu/%7Ehakim/LMR/docs/install.html#sample-data) from the [Hakim's UW Atmospheric Science page](https://www.atmos.uw.edu/~hakim/lmr/LMRv2/index.html), they will need to be pre-processed. The pickle file was created using an older version of Python, hence you will need to open the files as ```pd.sparse_dataframe``` objects before converting them to ```.csv``` files to be used for the rest of this notebook. 

Firstly load metadata csv and isolate only the columns we need to make a _cfr_ ```ProxyDatabase``` object

In [2]:
# loading metadata

df_meta = pd.read_csv('./proxydb_meta.csv')
df_p2k_meta = df_meta[df_meta['Proxy ID'].str.contains('PAGES2kv2')]
df_p2k_meta.set_index('Proxy ID', inplace=True)

In [3]:
archive_data = df_p2k_meta[['Archive type', 'Proxy measurement', 'Lat (N)', 'Lon (E)', 'Elev']]
archive_data['Proxy ID'] = archive_data.index  # recover it as a column


Load in the data csv, which contains arrays for time and proxy value. This step takes all the PAGES2k data and resets the index.

In [4]:
# loading data (time and value)

df = pd.read_csv('./proxydb_data.csv')
col = df.columns

clist = []
for c in col:
    if ('PAGES2kv2' in c) or ('Year C.E.' in c):
        clist.append(c)

p2k = df[clist]
p2k = p2k.reset_index(drop=True)

### Standardizing Proxy IDs (pid)

The `cfr.ProxyDatabase` class expects short proxy IDs (e.g., `Ocn_148`) that follow a specific location-based naming convention. We create a function that converts the long-format PAGES2kv2 IDs into these standardized shortened IDs.


In [5]:
time_col = p2k.columns[0]

pids = []
times = []
values = []

for pid in p2k.columns[1:]:  # Skip the time column
        # Get the time series for this proxy
        proxy_data = p2k[[time_col, pid]].dropna()
        
        if len(proxy_data) > 0:  # Only process if we have data
            pids.append(pid)
            times.append(proxy_data[time_col].tolist())
            values.append(proxy_data[pid].tolist())

In [6]:
def shorten_pid(pids): 
    # Split by colon first to handle proxy measurements with underscores
    parts_by_colon = pids.split(':')[0]  # Take everything before the colon
    parts = parts_by_colon.split('_')
    
    # Find the part that contains the number
    number = None
    for part in parts:
        if part.isdigit():
            number = part
            break
        elif 'NAm_' in part:
            number = part.split('_')[1]
            break
        elif 'Asi_' in part:
            number = part.split('_')[1]
            break
        elif 'Arc_' in part:
            number = part.split('_')[1]
            break
        elif 'Ant_' in part:
            number = part.split('_')[1]
            break
        elif 'Ocn_' in part:
            number = part.split('_')[1]
            break
        elif 'Aus_' in part:
            number = part.split('_')[1]
            break
    
    # Extract region from the second part
    region = parts[1].split('-')[0]
    
    # Map the region
    region_mapping = {
        'Asia': 'Asi',
        'Ocean2kHR': 'Ocn',
        'Africa': 'Afr',
        'Afr': 'Afr',
        'NAm': 'NAm',
        'Arc': 'Arc',
        'Ant': 'Ant',
        'Aus': 'Aus',
        'SAm': 'SAm'
    }
    region = region_mapping.get(region, region)
    
    if number is None:
        # If we still haven't found a number, look for it specifically
        for part in parts:
            if any(char.isdigit() for char in part):
                number = ''.join(char for char in part if char.isdigit())
                break
    
    return f"{region}_{number}"

We apply the naming function to our metadata dataframe so that the new index is the shorted `pid` as is used by _cfr_.

In [7]:
short_pids = []

for p in pids:
    lil = shorten_pid(p)
    short_pids.append(lil)

df_p2k_meta_short = df_p2k_meta.rename(index=shorten_pid)
archive_data_new_idx = archive_data.rename(index=shorten_pid)


data_df = pd.DataFrame({
        'pid': short_pids,
        'time': times,
        'value': values
    })

Now we take the data dataframe with shortened pids and merge it with the metadata dataframe to combine all the columns necessary to make a ProxyDatabase object. 

In [8]:
archive_data_new_idx = archive_data_new_idx.reset_index(names=['pid'])

new_pdb = pd.merge(
        archive_data_new_idx,
        data_df,
        on='pid',
        how='inner'
)

### Assigning Proxy Type Labels (ptype)

Each record in the `cfr.ProxyDatabase` is tagged with a `ptype`, which is a combination of the archive and measured variable (e.g., `tree.d18O`, `marine.MgCa`). We generate these using a mapping based on archive and measurement metadata.
\
Using this, we want a new column called 'ptype'. To generate this, we use the function `create_ptype`. The mapping for this function was done by hand, observing records from _cfr_'s built in ProxyDatabase to get an exact match to the metadata from Tardif's pickle.  

In [9]:
def create_ptype(row):

    archive_mapping = {
        'Tree Rings': 'tree',
        'Corals and Sclerosponges': 'coral',
        'Lake Cores': 'lake',
        'Ice Cores': 'ice',
        'Bivalve': 'bivalve',
        'Speleothems': 'speleothem',
        'Marine Cores': 'marine'
    }

    measure_mapping = {
        'Sr_Ca': 'SrCa',
        'trsgi': 'TRW',
        'calcification': 'calc',
        'd18O': 'd18O',
        'dD': 'dD',
        'MXD': 'MXD',
        'density': 'MXD',
        'composite': 'calc',
        'massacum': 'accumulation',
        'thickness': 'varve_thickness',
        'melt': 'melt',
        'RABD660_670': 'reflectance',
        'X_radiograph_dark_layer': 'varve_property'
    }
    
    # Get the shortened archive type
    archive = archive_mapping.get(row['Archive type'], row['Archive type'].lower())
    proxy = measure_mapping.get(row['Proxy measurement'])
    
    # Combine with proxy measurement
    return f"{archive}.{proxy}"

We now apply the function to our full dataframe.

In [10]:
new_pdb['ptype'] = new_pdb.apply(create_ptype, axis=1)
new_pdb = new_pdb.drop(['Archive type', 'Proxy measurement', 'Proxy ID'], axis=1)

## Filtering by resolution (as done in LMR)

We know that LMRv2.1 used only annual and subannually resolved records. In order to do this, we will take our new dataframe that has all the right columns, and turn it into a ```cfr.ProxyDatabase``` object, so we can filter by resolution. 

In [11]:
#new_pdb = new_pdb[new_pdb['pid'] != 'Afr_012']

new_pdb = new_pdb.rename(columns={
    'Lat (N)': 'lat',
    'Lon (E)': 'lon',
    'Elev': 'elev'
})

In [12]:
# examine the dataframe

new_pdb

,pid,lat,lon,elev,time,value,ptype
0,Ocn_148,43.6561,290.1983,-30.0,"[1033.0, 1034.0, 1035.0, 1036.0, 1037.0, 1038....","[1.14, 1.06, 0.8, 0.69, 1.21, 1.4, 1.26, 0.85,...",bivalve.d18O
1,Ocn_170,-21.8333,114.1833,-6.0,"[1900.0, 1901.0, 1902.0, 1903.0, 1904.0, 1905....","[-29.80451, 10.97039, 28.05434, 21.96584, -2.7...",coral.calc
2,Ocn_174,-21.9000,113.9667,-6.0,"[1900.0, 1901.0, 1902.0, 1903.0, 1904.0, 1905....","[10.976, -4.39066, -3.39162, 16.9882, -9.2862,...",coral.calc
3,Ocn_073,20.8300,273.2600,-3.1,"[1773.0, 1774.0, 1775.0, 1776.0, 1777.0, 1778....","[-0.605009101918667, 0.11922153808142, -0.9671...",coral.calc
4,Ocn_173,-17.5167,118.9667,-6.0,"[1900.0, 1901.0, 1902.0, 1903.0, 1904.0, 1905....","[-3.65572, 15.4608, 6.62304, -20.1302, 14.6722...",coral.calc
...,...,...,...,...,...,...,...
567,NAm_139,49.7000,241.1000,2000.0,"[1512.0, 1513.0, 1514.0, 1515.0, 1516.0, 1517....","[0.87, 0.885, 0.893, 0.893, 0.87, 0.81, 0.826,...",tree.MXD
568,NAm_200,44.8000,252.1000,2820.0,"[1508.0, 1509.0, 1510.0, 1511.0, 1512.0, 1513....","[1.03, 0.928, 0.929, 1.052, 1.001, 0.991, 0.94...",tree.MXD
569,NAm_130,55.3000,282.2000,50.0,"[1352.0, 1353.0, 1354.0, 1355.0, 1356.0, 1357....","[0.98, 0.988, 0.956, 0.845, 1.053, 1.129, 1.07...",tree.MXD
570,NAm_199,41.3000,252.3000,3150.0,"[1401.0, 1402.0, 1403.0, 1404.0, 1405.0, 1406....","[1.042, 0.998, 0.953, 1.079, 1.057, 0.91, 1.03...",tree.MXD


In [13]:
# create the ProxyDatabase object using the correctly labeled column names

blank = cfr.ProxyDatabase()

lmr_pdb = blank.from_df(new_pdb,
    pid_column='pid', 
    lat_column='lat',  
    lon_column='lon',  
    elev_column = 'elev',
    time_column = 'time',
    value_column = 'value',
    ptype_column = 'ptype'

)

To filter by resolution, we use _cfr_'s default `cfr.proxydb.filter` function. We filter by 'dt', which is the median timestep between time values. The following cell is to ensure that there are no ```None``` type values when computing dt.

In [14]:
missing_dt = [pid for pid, r in lmr_pdb.records.items() if r.dt is None]
print(f"{len(missing_dt)} records missing dt")

1 records missing dt


In [15]:
missing = lmr_pdb.filter(by='pid', keys=missing_dt, mode='exact')
lmr_pdb = lmr_pdb - missing

We want dt <= 1.0, so we expand the range a little bit in case there are any records with slightly greater than annual resolution. 

In [16]:
# everything should be annual

filt = lmr_pdb.filter(by='dt', keys= (0.0,1.2))

for record in filt.records:
    pobj = filt[record]
    print(pobj.pid, pobj.dt)

Ocn_148 1.0
Ocn_170 1.0
Ocn_174 1.0
Ocn_073 1.0
Ocn_173 1.0
Ocn_171 1.0
Ocn_065 1.0
Ocn_158 1.0
Ocn_172 1.0
Ocn_167 1.0
Ocn_121 1.0
Ocn_131 1.0
Ocn_163 1.0
Ocn_069 1.0
Ocn_112 1.0
Ocn_061 1.0
Ocn_182 1.0
Ocn_129 1.0
Ocn_067 1.0
Ocn_160 1.0
Ocn_070 1.0
Ocn_155 1.0
Ocn_153 1.0
Ocn_084 1.0
Ocn_093 1.0
Ocn_141 1.0
Ocn_159 1.0
Ocn_109 1.0
Ocn_161 1.0
Ocn_101 1.0
Ocn_143 1.0
Ocn_096 1.0
Ocn_176 1.0
Ocn_072 1.0
Ocn_106 1.0
Ocn_125 1.0
Ocn_060 1.0
Ocn_157 1.0
Ocn_110 1.0
Ocn_077 1.0
Ocn_166 1.0
Ocn_120 1.0
Ocn_099 1.0
Ocn_083 1.0
Ocn_179 1.0
Ocn_123 1.0
Ocn_130 1.0
Ocn_079 1.0
Ocn_087 1.0
Ocn_115 1.0
Ocn_162 1.0
Ocn_068 1.0
Ocn_075 1.0
Ocn_178 1.0
Ocn_088 1.0
Ocn_074 1.0
Ocn_097 1.0
Ocn_111 1.0
Ocn_139 1.0
Ocn_062 1.0
Ocn_138 1.0
Ocn_119 1.0
Ocn_076 1.0
Ocn_086 1.0
Ocn_181 1.0
Ocn_066 1.0
Ocn_128 1.0
Ocn_080 1.0
Ocn_098 1.0
Ocn_095 1.0
Ocn_116 1.0
Ocn_154 1.0
Ocn_169 1.0
Ocn_107 1.0
Ocn_114 1.0
Ocn_104 1.0
Ocn_127 1.0
Ocn_091 1.0
Ocn_140 1.0
Ocn_108 1.0
Ocn_078 1.0
Ocn_082 1.0
Ocn_081 1.0
Ocn_

### (Optional) Check for sparse records that don't meet the 0.5 threshold.
LMRv2.1 uses the parameter ```valid_frac``` to remove records that don't span more than half of the full length of the record. If the database is cleaned up, chances are this will not be an issue. But it is good to run just in case to see what records get filtered out. Records will get dropped during the calibration step of the data assimilation workflow if they do not meet the overlap requirement as well. 

In [17]:
valid_frac = 0.5
filtered_records = []

for pid, record in filt.records.items():
    time = np.array(record.time)
    value = np.array(record.value)

    mask = ~np.isnan(time) & ~np.isnan(value)
    time = time[mask]
    value = value[mask]

    if len(time) < 2:
        continue

    dt = record.dt
    years = time.astype(int)
    year_range = years.max() - years.min() + 1
    coverage = len(years) / year_range if year_range > 0 else 0

    # Keep if:
    # - subannual or annual AND coverage ≥ 0.5
    # - OR manually specified pid
    if coverage >= valid_frac or pid == 'Ocn_148':
        filtered_records.append(record)

### Final ProxyDatabase Object

The final `ProxyDatabase` object contains only PAGES2kv2 records that:
- Are annual or subannual in resolution (dt ≤ 1.2 years)
- Have at least 50% coverage over their date range
- Include complete time and value arrays
- Are labeled with a standardized `ptype` format (e.g., `tree.d18O`, `marine.MgCa`)

This cleaned and filtered database is now ready for use in the _cfr_ framework for climate field reconstruction.


In [18]:
# remove dropped records from final database

final_pdb = cfr.ProxyDatabase()
final_pdb += filtered_records

## Save Proxydatabase as netCDF file

In [19]:
final_pdb.to_nc('./prev_data/lmr_pickle_pages2k.nc')

>>> Warning: this is an experimental feature.
>>> Warning: this is an experimental feature.


100%|██████████| 544/544 [00:15<00:00, 35.39it/s]


>>> Data before 1 CE is dropped for records: ['Arc_032', 'Arc_033', 'Ant_007', 'Ant_010', 'Ant_028', 'Arc_004', 'Arc_026', 'Arc_020', 'Arc_022', 'NAm_011', 'NAm_019', 'Arc_002', 'Eur_006', 'Eur_003'].
ProxyDatabase saved to: ./prev_data/lmr_pickle_pages2k.nc


## Summary and Output

We have constructed a cleaned, standardized proxy database containing PAGES2kv2 records. This database has been filtered by resolution and (optionally) coverage and formatted for direct use in the _cfr_ paleoclimate data assimilation workflows.

The result has been saved as a NetCDF file:
```bash
./prev_data/lmr_pickle_pages2k.nc

